In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding

In [2]:
df=pd.read_csv(r'train.csv')

In [3]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
df2 = pd.read_csv(r'test.csv')

In [5]:
df2

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df2.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [8]:
df = df.fillna(' ')

In [9]:
df2 = df2.fillna(' ')

In [10]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
df2.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [12]:
tf.__version__

'2.10.0'

In [13]:
df['containt'] = df['title'] + ' ' + df['author']

In [14]:
df

,id,title,author,text,label,containt
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
pStem = PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
#     print('orignal = ',stemmed_content)
    stemmed_content = stemmed_content.lower()
#     print('lower = ',stemmed_content)
    stemmed_content = stemmed_content.split()
#     print('split = ',stemmed_content)
    stemmed_content = [pStem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#     print('stem = ',stemmed_content)
    stemmed_content =' '.join(stemmed_content)
#     print('join = ',stemmed_content)
    return stemmed_content

In [17]:
df['containt'] = df.containt.apply(stemming)

In [18]:
X=df.containt.values
print(X)
X = np.asarray(X)
print(X[0])
len(X)
# np.maxlen(X)

['hous dem aid even see comey letter jason chaffetz tweet darrel lucu'
 'flynn hillari clinton big woman campu breitbart daniel j flynn'
 'truth might get fire consortiumnew com' ...
 'maci said receiv takeov approach hudson bay new york time michael j de la merc rachel abram'
 'nato russia hold parallel exercis balkan alex ansari'
 'keep f aliv david swanson']
hous dem aid even see comey letter jason chaffetz tweet darrel lucu


20800

In [19]:
Y = df.label.values

In [20]:
# vectorizor = TfidfVectorizer()
# X = vectorizor.fit(X)


onehot_repr=[one_hot(words,5000)for words in X] 
onehot_repr


[[743, 241, 4897, 113, 3666, 770, 1086, 4153, 1307, 2013, 2326, 2773],
 [2413, 4645, 2247, 1193, 4973, 876, 3847, 3683, 996, 2413],
 [1887, 3729, 3596, 4242, 478, 1183],
 [3879, 3319, 454, 4469, 824, 2648, 4694, 941],
 [2202, 4973, 4037, 2752, 643, 1640, 4973, 3539, 3449, 1706, 3092, 3403],
 [831,
  3920,
  2579,
  703,
  2527,
  1299,
  1483,
  139,
  1574,
  2081,
  2498,
  3150,
  3947,
  4900,
  3847,
  3683,
  847],
 [2063, 2063, 1, 1209, 794, 2303, 106, 3330, 2043, 1961, 1595, 244],
 [3508,
  4374,
  225,
  3413,
  2700,
  1856,
  129,
  3031,
  1032,
  400,
  618,
  2471,
  996,
  1832],
 [756, 732, 4008, 2630, 1299, 779, 3470, 1407, 1042, 3916, 1032, 400, 618],
 [1157,
  2310,
  4779,
  3206,
  4899,
  3585,
  1299,
  4777,
  1032,
  400,
  618,
  2287,
  1212,
  4630,
  4829],
 [2115, 712, 2329, 918, 1342, 1606, 449, 3147, 1299, 3672, 2616, 1627],
 [4219, 143, 2334, 4047, 3196, 1406, 4738, 4085, 4796, 2954],
 [437, 115, 936, 1886, 1416, 3875, 1876, 3304, 1366, 3631, 1090, 3977

In [21]:
print(onehot_repr[1])

[2413, 4645, 2247, 1193, 4973, 876, 3847, 3683, 996, 2413]


In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2013 2326 2773]
 [   0    0    0 ... 3683  996 2413]
 [   0    0    0 ... 4242  478 1183]
 ...
 [   0    0    0 ... 3776 3120 1625]
 [   0    0    0 ...   67  264 3289]
 [   0    0    0 ... 4456 3359   42]]


In [23]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2413,
       4645, 2247, 1193, 4973,  876, 3847, 3683,  996, 2413])

In [24]:
X_final = np.array(embedded_docs)

In [25]:
X_final.shape

(20800, 20)

In [26]:
Y.shape

(20800,)

In [27]:
Y = np.array(Y)

In [28]:
Y.shape

(20800,)

In [29]:
model=Sequential()
model.add(Embedding(5000,40,input_length=sent_length))
model.add(LSTM(150))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 150)               114600    
                                                                 
 dense (Dense)               (None, 1)                 151       
                                                                 
Total params: 314,751
Trainable params: 314,751
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_final, Y, test_size=0.23, random_state=42)

In [31]:
# X_train = np.array(X_train)

In [32]:
# X_test = np.array(X_test)

In [33]:
print(len(X_train[5]))

20


In [34]:
X_test.shape

(4784, 20)

In [35]:
X_train.shape

(16016, 20)

In [36]:
X_final.shape,Y.shape

((20800, 20), (20800,))

In [37]:
y_train.shape

(16016,)

In [38]:
y_test.shape

(4784,)

In [39]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
251/251 [==============================] - 12s 35ms/step - loss: 0.1974 - accuracy: 0.9075 - val_loss: 0.0422 - val_accuracy: 0.9883
Epoch 2/15
251/251 [==============================] - 8s 32ms/step - loss: 0.0218 - accuracy: 0.9943 - val_loss: 0.0297 - val_accuracy: 0.9910
Epoch 3/15
251/251 [==============================] - 8s 32ms/step - loss: 0.0078 - accuracy: 0.9982 - val_loss: 0.0292 - val_accuracy: 0.9921
Epoch 4/15
251/251 [==============================] - 8s 32ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0338 - val_accuracy: 0.9912
Epoch 5/15
251/251 [==============================] - 8s 32ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.0357 - val_accuracy: 0.9908
Epoch 6/15
251/251 [==============================] - 9s 38ms/step - loss: 2.8572e-04 - accuracy: 1.0000 - val_loss: 0.0364 - val_accuracy: 0.9910
Epoch 7/15
251/251 [==============================] - 11s 43ms/step - loss: 1.2344e-04 - accuracy: 1.0000 - val_loss: 0.0402 - val_accura

In [40]:
loss, acc =model.evaluate(X_test,y_test)

150/150 [==============================] - 1s 9ms/step - loss: 0.0519 - accuracy: 0.9891


In [41]:
print(acc)

0.989130437374115


In [42]:
print(loss)

0.051937226206064224


In [43]:
X_test[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,   91, 4383,
       2279, 1105, 1299, 3065, 2209, 3847, 3860, 3662,  646])

In [44]:
y_test[3]

0

In [51]:
y_pred = model.predict(X_test)

150/150 [==============================] - 1s 9ms/step


In [52]:
print(y_pred[0])
print(y_test[0])

[1.]
1


In [60]:

y_pred.shape, y_test.shape

# y_pred.reshape(:,0)

((4784, 1), (4784,))

In [55]:
print(y_pred[0])
print(y_test[0])

[1.]
1
